In [1]:
import sys
import os

current_dir = os.getcwd()  # 현재 작업 디렉토리
parent_dir = os.path.dirname(current_dir)  # 상위 디렉토리
DATA_PATH = os.path.join(parent_dir, "data")
sys.path.append(parent_dir)

In [2]:
import os
import glob
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import ast


def safe_string_to_list(input_string, column_name):
    """
    안전하게 문자열을 리스트로 변환 (JSON이나 Python 리스트 형식 모두 처리 가능)

    Args:
        input_string (str): 리스트 형식의 문자열
        column_name (str): 현재 컬럼 이름

    Returns:
        list: 변환된 리스트 (변환 실패 시 빈 리스트 반환)
    """
    try:
        # 문자열을 리스트로 안전하게 변환
        input_list = ast.literal_eval(input_string)

        # 특정 컬럼에 대해 추가 처리
        if column_name == "diner_menu_price":
            # 빈 문자열 제외하고 정수로 변환
            return [int(x) for x in input_list if x != ""]
        else:
            return [x for x in input_list if x != ""]

    except (ValueError, SyntaxError, TypeError):
        # 변환 실패 시 빈 리스트 반환
        return []


diner_data_paths = glob.glob(os.path.join(DATA_PATH, "diner", "*.csv"))
review_data_paths = glob.glob(os.path.join(DATA_PATH, "review", "*.csv"))

# set cpu or cuda for default option
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device.type)


diner_df = pd.read_csv(diner_data_paths[0])
review_df = pd.DataFrame()
for review_data_path in review_data_paths:
    review_df = pd.concat([review_df, pd.read_csv(review_data_path)], axis=0)

# 함수 적용
diner_df["diner_menu_name"] = diner_df["diner_menu_name"].apply(
    lambda x: safe_string_to_list(x, "diner_menu_name")
)
diner_df["diner_menu_price"] = diner_df["diner_menu_price"].apply(
    lambda x: safe_string_to_list(x, "diner_menu_price")
)
diner_df["diner_tag"] = diner_df["diner_tag"].apply(
    lambda x: safe_string_to_list(x, "diner_tag")
)
diner_df["diner_review_tags"] = diner_df["diner_review_tags"].apply(
    lambda x: safe_string_to_list(x, "diner_review_tags")
)

review_df["reviewer_review_cnt"] = (
    review_df["reviewer_review_cnt"]
    .fillna(0)  # null값을 0으로 대체
    .astype(str)  # 문자열로 변환
    .str.replace(",", "")  # 콤마 제거
    .astype(int)  # 정수로 변환
)

In [3]:
import torch
from torch_geometric.data import HeteroData
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import ToUndirected

# 필요한 컬럼 선택
review_df = review_df[
    [
        "reviewer_id",
        "diner_idx",
        "reviewer_review_score",
        "reviewer_avg",
        "badge_level",
        "reviewer_user_name",
    ]
]
# diner_df = diner_df[['diner_idx', 'diner_category_large', 'diner_category_middle',
#                      'diner_category_small', 'diner_review_avg', 'real_good_review_percent',
#                      'real_bad_review_percent']]


# 사용자 및 음식점 인덱스 생성
user_ids = sorted(list(review_df["reviewer_id"].unique()))
diner_ids = sorted(list(review_df["diner_idx"].unique()))

num_diners = len(diner_ids)
num_reviewers = len(user_ids)

user_id_map = {id: idx for idx, id in enumerate(user_ids)}
diner_id_map = {id: idx for idx, id in enumerate(diner_ids)}

# 사용자와 음식점 ID를 각각 숫자 인덱스로 변환
review_df["reviewer_id"] = review_df["reviewer_id"].map(user_id_map)
review_df["diner_idx"] = review_df["diner_idx"].map(diner_id_map)
diner_df["diner_idx"] = diner_df["diner_idx"].map(diner_id_map)

/home/elinha/.pyenv/versions/3.11.1/envs/gnn_env/lib/python3.11/site-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /home/elinha/.pyenv/versions/3.11.1/envs/gnn_env/lib/python3.11/site-packages/torch_spline_conv/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(


2. 그래프 생성
PyTorch Geometric의 HeteroData 객체를 사용하여 그래프를 구성합니다.

2.1. HeteroData 객체 초기화

In [4]:
# PyG의 HeteroData 객체 생성
data = HeteroData()

# 노드 추가 (사용자와 음식점)
data["user"].num_nodes = len(user_ids)
data["restaurant"].num_nodes = len(diner_ids)


edge_index = torch.tensor(
    [review_df["reviewer_id"].values, review_df["diner_idx"].values], dtype=torch.long
)
data["user", "interacts", "restaurant"].edge_index = edge_index

# 엣지 추가 (리뷰 평점을 엣지 속성으로 사용)
data["user", "interacts", "restaurant"].edge_attr = torch.tensor(
    review_df["reviewer_review_score"].values, dtype=torch.float32
)
data["restaurant", "rev_interacts", "user"].edge_index = edge_index[
    [1, 0]
]  # 역방향 엣지

# 음식점 노드 특성 추가
# TODO:
# # 카테고리 컬럼을 원-핫 인코딩
# diner_categories = pd.get_dummies(diner_df[['diner_category_middle', 'diner_category_small']])

# # 나머지 컬럼과 결합
# diner_features = pd.concat([diner_categories, diner_df[['diner_review_avg', 'real_good_review_percent', 'real_bad_review_percent']]], axis=1)

# # NaN 값을 0으로 대체 (또는 평균값으로 대체 가능)
# diner_features = diner_features.fillna(0).values

# # 텐서로 변환
# data['restaurant'].x = torch.tensor(diner_features, dtype=torch.float32)

# 'diner_category_middle' 원-핫 인코딩
diner_df["category_final"] = diner_df["diner_category_small"].fillna(
    diner_df["diner_category_middle"]
)

diner_category_middle_encoded = pd.get_dummies(diner_df["category_final"])


# 텐서로 변환하여 PyG의 그래프 데이터에 추가
diner_features = diner_category_middle_encoded.fillna(0).values
data["restaurant"].x = torch.tensor(diner_features, dtype=torch.float32)

# TODO: 카테고리 너무 세분화되어있다면 차원줄이기
# from sklearn.decomposition import PCA
# pca = PCA(n_components=10)
# diner_features_reduced = pca.fit_transform(diner_features)
# data['restaurant'].x = torch.tensor(diner_features_reduced, dtype=torch.float32)


# 사용자 노드 특성 추가: badge_level 및 reviewer_avg
user_features_df = (
    review_df.groupby("reviewer_id")
    .agg(
        {
            "badge_level": "first",  # 첫 번째 값 사용
            "reviewer_avg": "first",  # 첫 번째 값 사용
        }
    )
    .reindex(user_ids)
    .fillna(0)
)  # user_ids 순서로 재배열 및 NaN 채우기

# badge_level과 reviewer_avg를 합쳐서 노드 특성으로 변환
user_features = user_features_df.values
data["user"].x = torch.tensor(user_features, dtype=torch.float32)

# 그래프를 무방향으로 변환 (선택 사항)
# data = ToUndirected()(data)

/home/elinha/.pyenv/versions/3.11.1/envs/gnn_env/lib/python3.11/site-packages/torch/utils/_device.py:79: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return func(*args, **kwargs)


In [5]:
print(data)
print(f"User node features shape: {data['user'].x.shape}")
print(f"Restaurant node features shape: {data['restaurant'].x.shape}")
print(f"Edge index shape: {data['user', 'interacts', 'restaurant'].edge_index.shape}")

HeteroData(
  user={
    num_nodes=351022,
    x=[351022, 2],
  },
  restaurant={
    num_nodes=59203,
    x=[78044, 255],
  },
  (user, interacts, restaurant)={
    edge_index=[2, 1074862],
    edge_attr=[1074862],
  },
  (restaurant, rev_interacts, user)={ edge_index=[2, 1074862] }
)
User node features shape: torch.Size([351022, 2])
Restaurant node features shape: torch.Size([78044, 255])
Edge index shape: torch.Size([2, 1074862])


## RandomLinkSplit의 개요
RandomLinkSplit은 PyTorch Geometric(PG)에서 제공하는 데이터 분할(transform) 도구로, 그래프 데이터를 학습, 검증, 테스트 데이터로 분할할 때 사용됩니다. sklearn의 train_test_split과는 차별화된 점이 많습니다. 특히 그래프 구조를 유지하며 분할할 수 있다는 점이 큰 장점입니다.

### 기존 데이터 분할과 RandomLinkSplit의 차이
> 연결될 노드를 확률값으로 계산

|특징	|sklearn의 train_test_split|	RandomLinkSplit|
|---|----|------|
|기본 대상|	독립된 데이터 포인트 (행 단위로 분할)|	그래프의 엣지 (노드 간 연결)|
|연결 구조 고려 여부|	데이터 간의 관계(엣지)를 고려하지 않음|	그래프 구조(엣지, 노드 연결 정보)를 유지|
|검증 및 테스트 데이터|	각 데이터 포인트를 무작위로 선택|	그래프의 일부 엣지를 제거하여 검증 및 테스트 데이터로 사용|
|음성 샘플 처리|	음성 샘플(negative samples)은 따로 생성하지 않음|	음성 샘플(연결되지 않은 노드 쌍)을 자동 생성|
|주요 응용 분야|	독립적인 데이터셋 분석 (예: 이미지, 텍스트)|	그래프 기반의 예측 문제 (링크 예측, 노드 분류 등)|


In [6]:
from torch_geometric.transforms import RandomLinkSplit

# 변환기(transformer) 생성
transform = RandomLinkSplit(
    num_val=0.1,  # 검증 데이터 비율 (10%)
    num_test=0.1,  # 테스트 데이터 비율 (10%)
    is_undirected=False,  # 그래프가 무방향인 경우 True
    edge_types=("user", "interacts", "restaurant"),  # 분할할 엣지 타입
    rev_edge_types=("restaurant", "rev_interacts", "user"),  # 역방향 엣지 타입
    neg_sampling_ratio=1.0,  # 음성 샘플 비율
    add_negative_train_samples=True,  # 학습 세트에도 음성 샘플 추가
)

# 데이터 분할 수행
train_data, val_data, test_data = transform(data)

print("train_data", train_data)
print("val_data", val_data)
print("test_data", test_data)

train_data HeteroData(
  user={
    num_nodes=351022,
    x=[351022, 2],
  },
  restaurant={
    num_nodes=59203,
    x=[78044, 255],
  },
  (user, interacts, restaurant)={
    edge_index=[2, 859890],
    edge_attr=[859890],
    edge_label=[1719780],
    edge_label_index=[2, 1719780],
  },
  (restaurant, rev_interacts, user)={ edge_index=[2, 859890] }
)
val_data HeteroData(
  user={
    num_nodes=351022,
    x=[351022, 2],
  },
  restaurant={
    num_nodes=59203,
    x=[78044, 255],
  },
  (user, interacts, restaurant)={
    edge_index=[2, 859890],
    edge_attr=[859890],
    edge_label=[214972],
    edge_label_index=[2, 214972],
  },
  (restaurant, rev_interacts, user)={ edge_index=[2, 859890] }
)
test_data HeteroData(
  user={
    num_nodes=351022,
    x=[351022, 2],
  },
  restaurant={
    num_nodes=59203,
    x=[78044, 255],
  },
  (user, interacts, restaurant)={
    edge_index=[2, 967376],
    edge_attr=[967376],
    edge_label=[214972],
    edge_label_index=[2, 214972],
  },
  (r

### 노드의 차원이 다른지 아닌지 확인

In [7]:
# 사용자 노드 특성 차원 확인
user_feature_dim = data["user"].x.shape[1]
print(f"User node feature dimension: {user_feature_dim}")

# 음식점 노드 특성 차원 확인
restaurant_feature_dim = data["restaurant"].x.shape[1]
print(f"Restaurant node feature dimension: {restaurant_feature_dim}")

# 특성 차원 비교
if user_feature_dim == restaurant_feature_dim:
    print("Feature dimensions are the same.")
else:
    print("Feature dimensions are different.")

User node feature dimension: 2
Restaurant node feature dimension: 255
Feature dimensions are different.


3. GraphSAGE 모델 정의<br>
3.1. 필요한 모듈 임포트

In [8]:
import torch.nn as nn
from torch_geometric.nn import SAGEConv, HeteroConv

3.2. 모델 클래스 정의

In [20]:
class GraphSAGERecommendationModel(nn.Module):
    def __init__(
        self, user_feature_dim, restaurant_feature_dim, hidden_channels, embedding_dim
    ):
        super().__init__()

        # # 사용자와 음식점 임베딩 레이어
        # self.user_embedding = nn.Embedding(num_users, embedding_dim)
        # self.restaurant_embedding = nn.Embedding(num_restaurants, embedding_dim)

        # GraphSAGE 레이어 정의
        self.conv1 = HeteroConv(
            {
                ("user", "interacts", "restaurant"): SAGEConv(
                    (user_feature_dim, restaurant_feature_dim), hidden_channels
                ),
                ("restaurant", "rev_interacts", "user"): SAGEConv(
                    (restaurant_feature_dim, user_feature_dim), hidden_channels
                ),
            },
            aggr="mean",
        )

        self.conv2 = HeteroConv(
            {
                ("user", "interacts", "restaurant"): SAGEConv(
                    (hidden_channels, hidden_channels), hidden_channels
                ),
                ("restaurant", "rev_interacts", "user"): SAGEConv(
                    (hidden_channels, hidden_channels), hidden_channels
                ),
            },
            aggr="mean",
        )

        # 최종 임베딩을 위한 선형 변환
        self.lin_user = nn.Linear(hidden_channels, embedding_dim)
        self.lin_restaurant = nn.Linear(hidden_channels, embedding_dim)

    def forward(self, data):
        x_dict = {"user": data["user"].x, "restaurant": data["restaurant"].x}
        edge_index_dict = data.edge_index_dict

        # 첫 번째 GraphSAGE 레이어
        x_dict = self.conv1(x_dict, edge_index_dict)
        x_dict = {key: x.relu() for key, x in x_dict.items()}

        # 두 번째 GraphSAGE 레이어
        x_dict = self.conv2(x_dict, edge_index_dict)
        x_dict = {key: x.relu() for key, x in x_dict.items()}

        # 최종 임베딩
        user_emb = self.lin_user(x_dict["user"])
        restaurant_emb = self.lin_restaurant(x_dict["restaurant"])

        return user_emb, restaurant_emb

    def predict(self, user_indices, restaurant_indices, user_emb, restaurant_emb):
        user_vectors = user_emb[user_indices]
        restaurant_vectors = restaurant_emb[restaurant_indices]
        scores = (user_vectors * restaurant_vectors).sum(dim=1)
        return scores


# 데이터에서 노드 특성 차원 가져오기
user_feature_dim = data["user"].x.shape[1]  # 예: 2
restaurant_feature_dim = data["restaurant"].x.shape[1]  # 예: 250

print(f"User feature dimension: {user_feature_dim}")
print(f"Restaurant feature dimension: {restaurant_feature_dim}")

# 모델 생성
embedding_dim = 64  # 원하는 임베딩 차원
hidden_channels = 128  # 숨겨진 채널 수

model = GraphSAGERecommendationModel(
    user_feature_dim=user_feature_dim,
    restaurant_feature_dim=restaurant_feature_dim,
    hidden_channels=hidden_channels,
    embedding_dim=embedding_dim,
).to(device)

User feature dimension: 2
Restaurant feature dimension: 255


# 4. 모델 학습
## 4.1. 모델 및 옵티마이저 초기화(링크 예측)

In [21]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
num_epochs = 100


# Early Stopping 설정
patience = 7  # 성능 개선이 없을 경우 중단할 epoch 수
best_val_loss = float("inf")  # 초기 Best Loss
patience_counter = 0  # 성능 개선이 없는 epoch 수

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    user_emb, restaurant_emb = model(train_data)

    edge_label_index = train_data["user", "interacts", "restaurant"].edge_label_index
    edge_label = train_data["user", "interacts", "restaurant"].edge_label

    preds = model.predict(
        edge_label_index[0],  # 사용자 인덱스
        edge_label_index[1],  # 음식점 인덱스
        user_emb,
        restaurant_emb,
    )

    loss = loss_fn(preds, edge_label.float())
    loss.backward()
    optimizer.step()

    # 검증 손실 계산
    model.eval()
    with torch.no_grad():
        user_emb_val, restaurant_emb_val = model(val_data)
        val_edge_label_index = val_data[
            "user", "interacts", "restaurant"
        ].edge_label_index
        val_edge_label = val_data["user", "interacts", "restaurant"].edge_label

        val_preds = model.predict(
            val_edge_label_index[0],
            val_edge_label_index[1],
            user_emb_val,
            restaurant_emb_val,
        )

        val_loss = loss_fn(val_preds, val_edge_label.float())

    print(
        f"Epoch {epoch + 1}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}"
    )

    if val_loss.item() < best_val_loss:
        best_val_loss = val_loss.item()
        patience_counter = 0  # 성능이 개선되었으므로 초기화
        # 모델 저장
        best_model_state = model.state_dict()
    else:
        patience_counter += 1  # 성능 개선이 없으므로 카운터 증가
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

# 최적 모델 복원
model.load_state_dict(best_model_state)
print("Training completed with Early Stopping")

Epoch 1, Loss: 0.6933, Val Loss: 0.8528
Epoch 2, Loss: 0.7663, Val Loss: 1.0224
Epoch 3, Loss: 1.1178, Val Loss: 0.6990
Epoch 4, Loss: 0.6898, Val Loss: 0.7134
Epoch 5, Loss: 0.6970, Val Loss: 0.7110
Epoch 6, Loss: 0.6966, Val Loss: 0.7020
Epoch 7, Loss: 0.6879, Val Loss: 0.7021
Epoch 8, Loss: 0.6831, Val Loss: 0.7129
Epoch 9, Loss: 0.6791, Val Loss: 0.7345
Epoch 10, Loss: 0.6719, Val Loss: 0.7773
Early stopping triggered
Training completed with Early Stopping


4.2. 학습 데이터 준비

5.2. 모델 평가

In [11]:
# # 평가 모드로 설정
# model.eval()
# with torch.no_grad():
#     user_emb, restaurant_emb = model(data)

#     val_edge_index = data['user', 'interacts', 'restaurant'].test_edge_index  # 평가 데이터 엣지
#     y_true = val['reviewer_review_score'].values  # 실제 라벨로 'reviewer_review_score' 사용
#     threshold = 4  # 평점 4 이상을 긍정으로 간주
#     y_true_binary = (val['reviewer_review_score'].values >= threshold).astype(int)

#     # 예측 값 계산
#     y_pred = (user_emb[val_edge_index[0]] * restaurant_emb[val_edge_index[1]]).sum(dim=1)



In [12]:
# y_pred_numpy = y_pred.detach().cpu().numpy()

In [13]:
# import numpy as np


# # Mean Average Precision (mAP) 계산
# # def mean_average_precision(y_true, y_pred):
# #     y_true_sorted = y_true[np.argsort(-y_pred)]
# #     cumsum = np.cumsum(y_true_sorted)
# #     precision_at_i = cumsum / (np.arange(1, len(y_true) + 1))
# #     return np.sum(precision_at_i * y_true_sorted) / np.sum(y_true_sorted)

# import numpy as np

# def mean_average_precision(y_true, y_pred):
#     """
#     Mean Average Precision (mAP) 계산
#     Args:
#         y_true: 이진 정답 레이블 (0 또는 1로 구성된 numpy 배열)
#         y_pred: 예측 점수 (numpy 배열)
#     Returns:
#         mAP 값
#     """
#     # y_pred를 기준으로 내림차순 정렬
#     sorted_indices = np.argsort(-y_pred)
#     y_true_sorted = y_true[sorted_indices]  # y_true 정렬

#     # 정답 위치에서 Precision@k 계산
#     cumsum = np.cumsum(y_true_sorted)  # 누적 합
#     precision_at_k = cumsum / (np.arange(1, len(y_true_sorted) + 1))  # Precision 계산

#     # Average Precision 계산
#     return np.sum(precision_at_k * y_true_sorted) / np.sum(y_true_sorted) if np.sum(y_true_sorted) > 0 else 0


# # nDCG@K 계산
# def ndcg_k(y_true, y_pred, k):
#     idx = np.argsort(y_pred)[::-1][:k]
#     y_true_k = np.take(y_true, idx)
#     dcg = np.sum((2**y_true_k - 1) / np.log2(np.arange(2, k + 2)))
#     idcg = np.sum((2**np.sort(y_true)[::-1][:k] - 1) / np.log2(np.arange(2, k + 2)))
#     return dcg / idcg if idcg > 0 else 0


# # y_true 이진화
# threshold = 4
# y_true_binary = (val['reviewer_review_score'].values >= threshold).astype(int)

# # mAP 계산
# map_score = mean_average_precision(y_true_binary, y_pred.cpu().numpy())
# print(f"mAP: {map_score:.4f}")

# K = 30
# ndcg_score = ndcg_k(y_true, y_pred_numpy, K)
# print(f"nDCG@{K}: {ndcg_score:.4f}")

In [14]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# import numpy as np

# def evaluate_regression(y_true, y_pred):
#     """
#     회귀 평가 지표 계산
#     Args:
#         y_true: 실제 값 (numpy array)
#         y_pred: 예측 값 (numpy array)
#     Returns:
#         dict: MSE, RMSE, MAE, R2 값을 포함하는 딕셔너리
#     """
#     mse = mean_squared_error(y_true, y_pred)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(y_true, y_pred)
#     r2 = r2_score(y_true, y_pred)
#     return {
#         'MSE': mse,
#         'RMSE': rmse,
#         'MAE': mae,
#         'R2': r2
#     }

In [15]:
# # 평가 모드로 설정
# model.eval()
# with torch.no_grad():
#     user_emb, restaurant_emb = model(data)

#     # 평가 데이터의 엣지와 실제 평점
#     val_edge_index = data['user', 'interacts', 'restaurant'].test_edge_index
#     y_true = val['reviewer_review_score'].values  # 실제 평점 (1~5 범위)

#     # 예측 값 계산 (내적)
#     y_pred = (user_emb[val_edge_index[0]] * restaurant_emb[val_edge_index[1]]).sum(dim=1).cpu().numpy()

# # 회귀 지표 계산
# results = evaluate_regression(y_true, y_pred)
# print(f"Evaluation Results: {results}")


추가 고려사항
데이터셋이 큰 경우:

메모리 부족 문제가 발생할 수 있습니다.
이 경우 미니배치 학습이나 NeighborSampler를 활용하여 샘플링 기반의 학습을 고려해 보세요.
노드 특징 추가:

사용자나 음식점에 대한 추가적인 특징(예: 프로필 정보, 카테고리 등)이 있다면 모델에 포함시켜 성능을 향상시킬 수 있습니다.
- 배지 레벨, 리뷰 쓴 수, 리뷰 쓴 업종, 
하이퍼파라미터 튜닝:

임베딩 차원, 학습률, 레이어 수 등을 변경하여 모델의 성능을 최적화할 수 있습니다.
모델 저장 및 로드:

학습된 모델을 저장하여 이후에 재사용할 수 있습니다.

In [16]:
# # 모델 저장
# torch.save(model.state_dict(), 'graphsage_model.pth')

# # 모델 로드
# model.load_state_dict(torch.load('graphsage_model.pth'))


5.3. 특정 사용자에게 추천 생성

In [22]:
review_df[review_df["reviewer_user_name"] == "로기"]

,reviewer_id,diner_idx,reviewer_review_score,reviewer_avg,badge_level,reviewer_user_name
125107,37586,22331,5.0,4.2,4,로기
378098,101904,33593,2.0,3.6,21,로기
386191,101904,46708,3.0,3.6,21,로기
388912,101904,47941,5.0,3.6,21,로기
47521,169294,1876,5.0,5.0,36,로기
...,...,...,...,...,...,...
404829,169294,41522,5.0,5.0,35,로기
405917,169294,4919,5.0,5.0,35,로기
407228,169294,4280,5.0,5.0,35,로기
408017,169294,20213,5.0,5.0,35,로기
